In [ ]:
## Merge img_meta into the dataset json file 
## name+description under the key "caption", page under the key "url", src under the key "imgUrl", 
## image id under the key "image_id"

### Next step: see "select_hard_distractors" for ranking distractos by RE w.r.t Q 
### (with a bit length penalty/bonus to balance length bias)

In [1]:
import json, random, time
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import copy
import os
import re
from nltk.corpus import stopwords
import pickle
import pylcs
np.set_printoptions(precision=4)

In [2]:
txt_dataset_remove_clean_longimg_tmp_0817 = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_remove_clean_longimg_tmp_0817.json", "r"))

In [11]:
txt_dataset_remove_clean_longimg_tmp_0817['10']['img_negFacts'][0]

{'caption': 'Oscar Zariski. Oscar Zariski (1899–1986)',
 'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/e/e5/Oscar_Zariski.jpg',
 'scores': '(0.05, 0.0, 0.0, 0.05, 0.05)',
 'title': 'Oscar Zariski',
 'url': 'https://en.wikipedia.org/wiki/Oscar_Zariski'}

In [6]:
img_meta = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/img_metadata-Copy1.json", "r"))

In [12]:
img_meta['0'].keys()

dict_keys(['src', 'page', 'name', 'description'])

In [10]:
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0812.json", "r"))

In [27]:
img_dataset_merge_meta_tmp_0817 = {}
count = 0
for k in img_dataset:
    img_dataset_merge_meta_tmp_0817[k] = copy.deepcopy(img_dataset[k])
    img_dataset_merge_meta_tmp_0817[k]['img_posFacts'] = []
    img_dataset_merge_meta_tmp_0817[k]['img_negFacts'] = []
    for g in img_dataset[k]['GoldIds']:
        img_meta_key = str(int(g))
        m = img_meta[img_meta_key]
        img_dataset_merge_meta_tmp_0817[k]['img_posFacts'].append({
            'image_id': g,
            'title': m['name'].replace("_", " ").replace(".JPG", " ").strip(),
            'caption': (m['name'] + m['description']).replace("_", " ").replace(".JPG", " ").strip(),
            'url': m['page'],
            'imgUrl': m['src']
        })
    for g in img_dataset[k]['DistractorIds']:
        img_meta_key = str(int(g))
        m = img_meta[img_meta_key]
        img_dataset_merge_meta_tmp_0817[k]['img_negFacts'].append({
            'image_id': g,
            'title': m['name'].replace("_", " ").replace(".JPG", " ").strip(),
            'caption': (m['name'] + m['description']).replace("_", " ").replace(".JPG", " ").strip(),
            'url': m['page'],
            'imgUrl': m['src']
        })
    del img_dataset_merge_meta_tmp_0817[k]['GoldIds']
    del img_dataset_merge_meta_tmp_0817[k]['DistractorIds']
    count += 1
    if count%5000 == 0: print(count)

print(len(img_dataset_merge_meta_tmp_0817))
json.dump(img_dataset_merge_meta_tmp_0817, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_merge_meta_tmp_0817.json", "w"), indent=4)


5000
10000
15000
20000
25000
25452


In [28]:
pprint(img_dataset_merge_meta_tmp_0817['3408'])

{'A': '"The wrestler with the single kneepad wore cornrows."',
 'DistractorFacts': [{'fact': 'In the qualifying tournament for the 2016 Rio '
                              'Olympics held in Istanbul, she won in the final '
                              'round beating Polish wrestler Iwona Matkowska, '
                              'and qualified for the Olympics in the process.',
                      'scores': '(0.13, 0.0, 0.13)',
                      'title': 'Vinesh Phogat',
                      'url': 'https://en.wikipedia.org/wiki/Vinesh_Phogat'},
                     {'fact': 'Athletes from 87 nations earned medals at the '
                              '2016 Summer Olympics, breaking the record of '
                              'most nations winning a medal at a single '
                              'edition of the Olympics.',
                      'scores': '(0.1, 0.0, 0.1)',
                      'title': '2016 Summer Olympics medal table',
                      'url': 'ht